# One-way Function
## Produced by: Ruben Girela Castellón

In [2]:
import numpy as np
import math
import random

# ========================================================================
# Functions of Modular Arithmetic
# ========================================================================

'''
función que calcula el maximo comun divisor y devuelve el mcd y u, v 
pertenecientes a numeros enteros.
'''
def mcd(x, y, u=[1,0], v=[0,1]):
    
    #Los pongo en valor absoluto
    dividendo = abs(x)
    divisor = abs(y)
            
    #calculo el cociente y el resto
    cociente, resto = divmod(dividendo, divisor)
    
    #si el resto es 0 termina y devuelve el cmd, u y v
    if(resto == 0): return divisor, u[-1], v[-1]
    
    '''
    en caso contrario copio la listas de u y v y añado el nuevo valor u y v
    a traves de la formula: u_i = u_(i-2) - q_i * u_(i-1), siendo q_i el cociente
    calculado.
    '''
    u1 = u.copy()
    u1.append(u1[-2]-cociente*u1[-1])
    v1 = v.copy()
    v1.append(v1[-2]-cociente*v1[-1])
    
    #y repetimos el proceso, hasta que el resto sea 0
    resultado = mcd(divisor,resto, u1, v1)
    
    '''
    esto se hace ya que es una función recursiva y tengo que ir pasando el 
    resultado en cada iteración recursiva.
    '''
    return resultado

#FUncion que dado un numero x, determina si x es probablemente primo usando el metodo Miller-Rabin
def isPrime(x):
    
    #por si el valor que recibe no es entero devuelve 0 (no es primo)
    if(type(x) != type(int())):
       return 0
    
    #si el numero x < 5
    if(x < 5):
        #y son 2 o 3 son probablemente primos
        if(x==2 or x ==3):
            return 1
        
        #en caso contrario no lo seran
        return 0
    
    #si el numero es par directamente no es primo
    if(x%2 == 0):
        return 0
    
    #para calcular s y u inicialmente valdran s = x - 1 y u = 0
    s = x-1
    u = 0
    
    #mientras s sea par
    while(s%2 == 0):
        #incremento u en 1
        u += 1
        #y divido s a la mitad entera
        s = s // 2
    
    #para saber si es primo o no inicialmente el resultado valdra 1
    result = 1
    
    #contador k que contara las veces que lo comprueba, en mi caso 10 como mucho
    k = 0
    
    #por defecto recorrera 10 veces
    maximo = 10
    
    '''
    pero si el numero es pequeño, por ejemplo 5, recorrera menos de 10 veces, 
    ya que estaríamos repitiendo numeros ya calculados anteriormente
    '''
    if(x-3 < 10):
        maximo = x-3
    
    #genero una lista de n numeros aleatorios entre [2, x-2] no repetidos
    lista = []#random.sample(range(2,x-1),maximo)
    
    
    #mientras el resultado es 1 o x-1 (-1) y no ha llegado al numero maximo de veces
    while((result == 1 or result == x-1) and k < maximo):
        
        #obtengo aleatoriamente otro numero
        #a = lista.pop()
        
        #obtengo un valor aleatorio
        a = random.randint(2,x-1)
        '''
        compruebo que no es repetido, si lo es calculo 
        otro valor, hasta que no sea un valor repetido
        '''
        while(a in lista):
            a = random.randint(2,x-1)
        #y lo añado a la lista
        lista.append(a)
        
        #contador que regulara el numero de calculos
        contador = 0
        
        #se hace el primer calculo a^s mod x
        result = pow(a,s,x)
        
        #si el resultado no es 1 o -1 (x-1) y el contador es < u
        while((result != 1 and result != x-1) and contador < u):
            #incremento el contador
            contador += 1
            #calculo a^s·2^k mod x
            result = pow(result,pow(2,contador),x)
            
            '''
                si a^(2^(k-1) * s) = 1, compruebo el siguiente:
                a^(2^(k) * s) es != de 1 o -1
            '''
            if((result == 1) and contador+1 < u):
                contador += 1
                result = pow(result, pow(2,contador),x)
           
        k += 1 #incremento el numero de veces
        
    #si ha llegado al numero maximo de veces y el resultado sigue siendo 1 o -1 (x-1)
    if(k== maximo and (result == 1 or result == x-1)):
        return 1 #es primo
    
    #en caso contrario no es primo
    return 0

#función que calcula a^(-1) mod b, para cualquier a, b enteros que sean primos relativos
def inversa(x, y):
    
    #para que los valores sean enteros
    x = int(x)
    y = int(y)
    
    #calculo el mcd, la v y u
    divisor, u, v = mcd(x,y)
    
    '''
    Compruebo solo el 1, ya que el divisor y el dividendo los convierto en valor 
    absoluto, con lo cual incluye tambien el -1.
    '''
    #si el divisor es 1 son primos relativos.
    if(divisor == 1):
        #calculo a^(-1) su inversa haciendo u mod b
        return u % y        
    
    #si no son primos relativos devuelvo -1 e imprimo un mensaje de error
    print("Error no son primos relativos, ya que ambos son divisibles por", divisor)
    return -1

## Backpack function (Knapsack) and its inverse

We are going to use the case of the super-increasing sequence of positive numbers, for the **knappsack problem**.

To solve this problem, we use the Greedy algorithm, which consists of:

Given a sequence of values **a** and a maximum allowed size **n**, we are going to:

- Initialize a vector **b** to 0 with the same length as the sequence **a**.
- For i = {k, k-1, ..., 1, 0};  where **k** is the lenght of **a** - 1.
    - Compare if $a_i \leq n$:
        - Change the value $b_i=1$
        - $n = n - a_i$
    - And we repeat the process until n = 0:
        - return b

In [1]:
def mochilaGreedyCreciente(n,a):
    #creo el vector de soluciones
    r = np.zeros(len(a),dtype=int)
    
    #recorro los valores del conjunto a de forma decreciente
    for i in np.arange(len(a))[::-1]:
        
        #si el valor es <= a la capacidad máxima permitida
        if(a[i] <= n):
            
            #pongo un 1 de que se mete ese valor
            r[i] = 1
            # y le restamos a la capacidad máxima ese valor
            n -= a[i]
            
            #si la capacidad máxima es 0 termina y devuelve la solución
            if(n == 0):
                return r
    
    return r

Now with the algorithm Greedy we are going to implement the first practical public key criptosistem even proposed.

It was invented by **Merkle** and **Hellman**, shortly after **Diffie** and **Hellman** established the basic principles of public key criptography. 

Currently it is not used, since the keys obtained by disguising supercreasing sequences in this way are believed to be special and using the knapsack problem can be very easy.

### Cipher

Given a super-increasing sequence of positive numbers $(a_0, a_1, ..., a_k)$, where $\sum_{i=0}^{l}(a_i) < a_j; \thinspace j=l+1$ and the message **m** = a sequence of 0s and/or 1s of length **k**, where **k** is the length of the sequence **a**.

We generate 2 prime positive random numbers **n** and **u**, such that $ n > \sum_{i=0}^{k}a_i$ and **gcd(n, u) = 1**.

We generate with these values another sequence **a*** of the same length as **a**, which will contain the values $a^*_i = u * a_i \thinspace mod \thinspace n$.

And finally we generate $c = \sum_{i=0}^{k} a^*_i * m_i$, where **c** is the encrypted message.

### Decipher

Given a super-increasing sequence of positive numbers $(a_0, a_1, ..., a_k)$, the cipher message **c**, and the random values **n** and **u** generated in the cipher functión.